In [5]:
from ismn.interface import ISMN_Interface
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import re
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load ISMN data

In [2]:
path=r'data\africa\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.zip'

In [3]:
# Load data and metadata
ismn_data=ISMN_Interface(path,parallel=True)
ismn_data

Using the existing ismn metadata in data\africa\python_metadata\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.csv to set up ISMN_Interface. 
If there are issues with the data reader, you can remove the metadata csv file to repeat metadata collection.


ismn.base.IsmnRoot Zip at data\africa\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.zip
with Networks[Stations]:
------------------------
  AMMA-CATCH: ['Banizoumbou', 'Belefoungou-Mid', 'Belefoungou-Top', 'Nalohou-Mid', 'Nalohou-Top', 'Tondikiboro', 'Wankama'],
  SD_DEM: ['Demokeya'],
  TAHMO: ['AtebubuSHS,Atebubu', 'BiaSHTSDebiso', 'BimbillaSHS,Bimbilla', 'BongoSHS,Bongo', 'BuiPowerAuthority,Bui', 'CRIG(SoilMoistureStation1)', 'CRIG(SoilMoistureStation2)', 'CSIR-SARI,Nyankpala-Tamale', 'CSIR-SARI,Tanina', 'CoteDIvoireMeteoOffice', 'DaffiamaSHS,Daffiama', 'DedanKimathiUniversityofTechnology', 'GbewaaCollegeofEducation,Pusiga', 'HanSHS,Han', 'ImurtottPrimaryschool', 'JuabosoSHS', 'KNUSTFarm,Kumasi', 'KajajiSHS,Kajaji', 'KpandaiSHS,Kpandai', 'MagomanoSecondarySchool', 'MahaliMzuri', 'MaraTriangleAirstrip', 'MasenoUniversity', 'MurungaruSecondarySchool', 'NDMA-TanaRiverHola', 'NaboishoCamp', 'NotreDameSeminary,SHS,Navrongo', 'SacredHeartSHSNsoatre', 'SenchuraSecondaryS

### Explore data

In [ ]:
# Select 'Network' and 'Station', print metadata
sm=ismn_data['AMMA-CATCH'].to_xarray(variable='soil_moisture')
sm


100%|██████████| 7/7 [00:01<00:00,  6.82it/s]


<xarray.Dataset> Size: 9MB
Dimensions:                  (sensor: 16, date_time: 22344)
Coordinates:
  * date_time                (date_time) datetime64[ns] 179kB 2016-06-14 ... ...
Dimensions without coordinates: sensor
Data variables: (12/25)
    soil_moisture            (sensor, date_time) float64 3MB dask.array<chunksize=(1, 12348), meta=np.ndarray>
    soil_moisture_flag       (sensor, date_time) object 3MB dask.array<chunksize=(1, 12348), meta=np.ndarray>
    soil_moisture_orig_flag  (sensor, date_time) object 3MB dask.array<chunksize=(1, 12348), meta=np.ndarray>
    depth_from               (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    depth_to                 (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    clay_fraction            (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    ...                       ...
    saturation               (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    silt_fraction            (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    station                  (sensor) <U15 960B dask.array<chunksize=(1,), meta=np.ndarray>
    timerange_from           (sensor) datetime64[ns] 128B dask.array<chunksize=(1,), meta=np.ndarray>
    timerange_to             (sensor) datetime64[ns] 128B dask.array<chunksize=(1,), meta=np.ndarray>
    variable                 (sensor) <U13 832B dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    n_sensors:   16
    n_stations:  7
    network:     AMMA-CATCH

In [21]:
print("=== AMMA-CATCH Dataset Overview ===")
print(f"Dataset dimensions: {sm.dims}")
print(f"Number of sensors: {sm.dims['sensor']}")
print(f"Time range: {sm.date_time.min().values} to {sm.date_time.max().values}")
print(f"Total time points: {sm.dims['date_time']}")
print("\nData variables:")
for var in sm.data_vars:
    print(f"  - {var}: {sm[var].dims}")

=== AMMA-CATCH Dataset Overview ===
Dataset dimensions: FrozenMappingWarningOnValuesAccess({'sensor': 16, 'date_time': 22344})
Number of sensors: 16
Time range: 2016-06-14T00:00:00.000000000 to 2018-12-31T23:00:00.000000000
Total time points: 22344

Data variables:
  - soil_moisture: ('sensor', 'date_time')
  - soil_moisture_flag: ('sensor', 'date_time')
  - soil_moisture_orig_flag: ('sensor', 'date_time')
  - depth_from: ('sensor',)
  - depth_to: ('sensor',)
  - clay_fraction: ('sensor',)
  - climate_KG: ('sensor',)
  - climate_insitu: ('sensor',)
  - elevation: ('sensor',)
  - instrument: ('sensor',)
  - latitude: ('sensor',)
  - lc_2000: ('sensor',)
  - lc_2005: ('sensor',)
  - lc_2010: ('sensor',)
  - lc_insitu: ('sensor',)
  - longitude: ('sensor',)
  - network: ('sensor',)
  - organic_carbon: ('sensor',)
  - sand_fraction: ('sensor',)
  - saturation: ('sensor',)
  - silt_fraction: ('sensor',)
  - station: ('sensor',)
  - timerange_from: ('sensor',)
  - timerange_to: ('sensor',)
 

In [30]:
print("\n=== Extracting Static Information ===")

# Create a dataframe with static information for each sensor
static_data = []

for sensor_idx in range(sm.dims['sensor']):
    sensor_data = {
        'sensor_id': sensor_idx,
        'network': sm.network.values[sensor_idx] if 'network' in sm.variables else 'AMMA-CATCH',
        'station': sm.station.values[sensor_idx] if 'station' in sm.variables else f'Station_{sensor_idx}',
        'sensor_name': str(sm.instrument.values[sensor_idx]) if 'instrument' in sm.variables else f'Sensor_{sensor_idx}',
        'latitude': float(sm.latitude.values[sensor_idx]),
        'longitude': float(sm.longitude.values[sensor_idx]),
        'elevation': float(sm.elevation.values[sensor_idx]) if 'elevation' in sm.variables else np.nan,
        'depth_from': float(sm.depth_from.values[sensor_idx]) if 'depth_from' in sm.variables else np.nan,
        'depth_to': float(sm.depth_to.values[sensor_idx]) if 'depth_to' in sm.variables else np.nan,
        'clay_fraction': float(sm.clay_fraction.values[sensor_idx]) if 'clay_fraction' in sm.variables else np.nan,
        'sand_fraction': float(sm.sand_fraction.values[sensor_idx]) if 'sand_fraction' in sm.variables else np.nan,
        'silt_fraction': float(sm.silt_fraction.values[sensor_idx]) if 'silt_fraction' in sm.variables else np.nan,
        'organic_carbon': float(sm.organic_carbon.values[sensor_idx]) if 'organic_carbon' in sm.variables else np.nan,
    }
    static_data.append(sensor_data)

static_df = pd.DataFrame(static_data)
print(f"Static data extracted for {len(static_df)} sensors")
print("\nSample of static data:")
print(static_df.head())


=== Extracting Static Information ===
Static data extracted for 16 sensors

Sample of static data:
   sensor_id     network          station sensor_name  latitude  longitude  \
0          0  AMMA-CATCH      Banizoumbou     CS616-1  13.53250    2.66040   
1          1  AMMA-CATCH      Banizoumbou     CS616-2  13.53250    2.66040   
2          2  AMMA-CATCH  Belefoungou-Mid       CS616   9.79506    1.70994   
3          3  AMMA-CATCH  Belefoungou-Mid       CS616   9.79506    1.70994   
4          4  AMMA-CATCH  Belefoungou-Top       CS616   9.78986    1.70994   

   elevation  depth_from  depth_to  clay_fraction  sand_fraction  \
0      208.0        0.05      0.05            5.0           90.0   
1      208.0        0.05      0.05            5.0           90.0   
2      414.0        0.05      0.05           22.0           49.0   
3      414.0        0.10      0.10           22.0           49.0   
4      414.0        0.05      0.05           22.0           49.0   

   silt_fraction  orga

In [31]:
# 3. CALCULATE DAILY MEAN SOIL MOISTURE
# ========================================

print("\n=== Calculating Daily Mean Soil Moisture ===")

# Convert time to pandas datetime for easier resampling
sm_with_time = sm.assign_coords(date_time=pd.to_datetime(sm.date_time.values))

# Calculate daily means for soil moisture
daily_sm = sm_with_time.soil_moisture.resample(date_time='D').mean()

print(f"Daily soil moisture data shape: {daily_sm.shape}")
print(f"Date range: {daily_sm.date_time.min().values} to {daily_sm.date_time.max().values}")


=== Calculating Daily Mean Soil Moisture ===
Daily soil moisture data shape: (16, 931)
Date range: 2016-06-14T00:00:00.000000000 to 2018-12-31T00:00:00.000000000


In [32]:
# ========================================
# 4. CREATE COMPREHENSIVE DATAFRAME
# ========================================

print("\n=== Creating Comprehensive DataFrame ===")

# Create a long-format dataframe with all data
all_data = []

# Get the number of sensors from the daily resampled data
n_sensors = daily_sm.shape[0] if len(daily_sm.shape) > 1 else daily_sm.sizes['sensor']

for sensor_idx in range(n_sensors):
    sensor_static = static_df.iloc[sensor_idx]
    
    # Get daily soil moisture values for this sensor
    sensor_sm_data = daily_sm.isel(sensor=sensor_idx)
    
    # Calculate statistics with proper data filtering
    sm_values = sensor_sm_data.values
    
    # Filter out invalid values: NaN, negative values, and common missing value codes
    # Soil moisture should be between 0 and 1 (or 0-100 if in percentage)
    # Common missing value codes: -9999, -999, values < -1
    valid_mask = (
        ~np.isnan(sm_values) &  # Remove NaN
        (sm_values >= 0) &      # Remove negative values
        (sm_values <= 1) &      # Remove unrealistic high values (assuming 0-1 range)
        (sm_values != -9999) &  # Remove common missing value code
        (sm_values != -999)     # Remove another common missing value code
    )
    
    valid_values = sm_values[valid_mask]
    
    if len(valid_values) > 0:
        stats = {
            'sm_mean': np.mean(valid_values),
            'sm_std': np.std(valid_values),
            'sm_min': np.min(valid_values),
            'sm_max': np.max(valid_values),
            'sm_median': np.median(valid_values),
            'data_count': len(valid_values),
            'data_coverage': len(valid_values) / len(sm_values) * 100,
            'invalid_count': len(sm_values) - len(valid_values),
            'original_range_min': np.nanmin(sm_values),
            'original_range_max': np.nanmax(sm_values)
        }
    else:
        stats = {
            'sm_mean': np.nan, 'sm_std': np.nan, 'sm_min': np.nan,
            'sm_max': np.nan, 'sm_median': np.nan, 'data_count': 0, 
            'data_coverage': 0, 'invalid_count': len(sm_values),
            'original_range_min': np.nanmin(sm_values),
            'original_range_max': np.nanmax(sm_values)
        }
    
    # Combine static and statistical data
    sensor_complete = {**sensor_static.to_dict(), **stats}
    all_data.append(sensor_complete)

# Create the final comprehensive dataframe
final_df = pd.DataFrame(all_data)

print(f"Final dataframe shape: {final_df.shape}")
print("\nColumns in final dataframe:")
for col in final_df.columns:
    print(f"  - {col}")



=== Creating Comprehensive DataFrame ===
Final dataframe shape: (16, 23)

Columns in final dataframe:
  - sensor_id
  - network
  - station
  - sensor_name
  - latitude
  - longitude
  - elevation
  - depth_from
  - depth_to
  - clay_fraction
  - sand_fraction
  - silt_fraction
  - organic_carbon
  - sm_mean
  - sm_std
  - sm_min
  - sm_max
  - sm_median
  - data_count
  - data_coverage
  - invalid_count
  - original_range_min
  - original_range_max


In [33]:
# DATA QUALITY ASSESSMENT
# ========================================

print("\n=== Data Quality Assessment ===")
print(f"Total invalid values removed across all sensors: {final_df.invalid_count.sum()}")
print(f"Average data coverage per sensor: {final_df.data_coverage.mean():.1f}%")
print(f"Sensors with >50% valid data: {(final_df.data_coverage > 50).sum()}")
print(f"Original data range: {final_df.original_range_min.min():.2f} to {final_df.original_range_max.max():.2f}")
print(f"Cleaned data range: {final_df.sm_min.min():.4f} to {final_df.sm_max.max():.4f}")
print("\nData coverage by sensor:")
for idx, row in final_df.iterrows():
    print(f"  {row.sensor_name}: {row.data_coverage:.1f}% ({row.data_count} valid / {row.data_count + row.invalid_count} total)")

# =======


=== Data Quality Assessment ===
Total invalid values removed across all sensors: 3390
Average data coverage per sensor: 77.2%
Sensors with >50% valid data: 16
Original data range: -9999.00 to 0.32
Cleaned data range: 0.0054 to 0.3166

Data coverage by sensor:
  CS616-1: 100.0% (931 valid / 931 total)
  CS616-2: 100.0% (931 valid / 931 total)
  CS616: 54.6% (508 valid / 931 total)
  CS616: 54.6% (508 valid / 931 total)
  CS616: 60.3% (561 valid / 931 total)
  CS616: 60.3% (561 valid / 931 total)
  CS616: 56.6% (527 valid / 931 total)
  CS616: 56.6% (527 valid / 931 total)
  CS616: 50.6% (471 valid / 931 total)
  CS616: 50.6% (471 valid / 931 total)
  CS616: 96.6% (899 valid / 931 total)
  CS616-1: 99.1% (923 valid / 931 total)
  CS616-2: 99.1% (923 valid / 931 total)
  CS616: 98.9% (921 valid / 931 total)
  CS616-1: 99.0% (922 valid / 931 total)
  CS616-2: 99.0% (922 valid / 931 total)


In [34]:
final_df

,sensor_id,network,station,sensor_name,latitude,longitude,elevation,depth_from,depth_to,clay_fraction,...,sm_mean,sm_std,sm_min,sm_max,sm_median,data_count,data_coverage,invalid_count,original_range_min,original_range_max
0,0,AMMA-CATCH,Banizoumbou,CS616-1,13.53250,2.66040,208.0,0.05,0.05,5.0,...,0.027932,0.028664,0.005450,0.127196,0.009679,931,100.000000,0,0.005450,0.127196
1,1,AMMA-CATCH,Banizoumbou,CS616-2,13.53250,2.66040,208.0,0.05,0.05,5.0,...,0.039079,0.032553,0.017350,0.162788,0.019387,931,100.000000,0,0.017350,0.162788
2,2,AMMA-CATCH,Belefoungou-Mid,CS616,9.79506,1.70994,414.0,0.05,0.05,22.0,...,0.145056,0.091609,0.026675,0.290517,0.178408,508,54.564984,423,-9999.000000,0.290517
3,3,AMMA-CATCH,Belefoungou-Mid,CS616,9.79506,1.70994,414.0,0.10,0.10,22.0,...,0.167152,0.093208,0.054354,0.316633,0.195223,508,54.564984,423,-9999.000000,0.316633
4,4,AMMA-CATCH,Belefoungou-Top,CS616,9.78986,1.70994,414.0,0.05,0.05,22.0,...,0.095050,0.075493,0.006929,0.217808,0.087933,561,60.257787,370,-5832.735192,0.217808
5,5,AMMA-CATCH,Belefoungou-Top,CS616,9.78986,1.70994,414.0,0.10,0.10,22.0,...,0.168633,0.076502,0.084658,0.297483,0.153650,561,60.257787,370,-5832.707038,0.297483
6,6,AMMA-CATCH,Nalohou-Mid,CS616,9.74530,1.60530,441.0,0.05,0.05,22.0,...,0.110112,0.082414,0.014679,0.250525,0.092883,527,56.605800,404,-9999.000000,0.250525
7,7,AMMA-CATCH,Nalohou-Mid,CS616,9.74530,1.60530,441.0,0.10,0.10,22.0,...,0.132999,0.076507,0.045096,0.262229,0.099238,527,56.605800,404,-9999.000000,0.262229
8,8,AMMA-CATCH,Nalohou-Top,CS616,9.74407,1.60580,445.0,0.05,0.05,22.0,...,0.094512,0.053011,0.013929,0.227408,0.112004,471,50.590763,460,-9999.000000,0.227408
9,9,AMMA-CATCH,Nalohou-Top,CS616,9.74407,1.60580,445.0,0.10,0.10,22.0,...,0.104291,0.055329,0.020883,0.264008,0.120262,471,50.590763,460,-9999.000000,0.264008


In [35]:
# ========================================
# 5. CREATE GEODATAFRAME FOR SHAPEFILE
# ========================================

print("\n=== Creating GeoDataFrame for Shapefile Export ===")

# Create Point geometries from lat/lon
geometry = [Point(xy) for xy in zip(final_df.longitude, final_df.latitude)]

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(final_df, geometry=geometry, crs='EPSG:4326')

# Clean column names for shapefile compatibility (max 10 characters)
column_mapping = {
    'sensor_id': 'sensor_id',
    'network': 'network',
    'station': 'station',
    'sensor_name': 'sensor_nm',
    'latitude': 'lat',
    'longitude': 'lon',
    'elevation': 'elevation',
    'depth_from': 'depth_from',
    'depth_to': 'depth_to',
    'clay_fraction': 'clay_frac',
    'sand_fraction': 'sand_frac',
    'silt_fraction': 'silt_frac',
    'organic_carbon': 'org_carbon',
    'sm_mean': 'sm_mean',
    'sm_std': 'sm_std',
    'sm_min': 'sm_min',
    'sm_max': 'sm_max',
    'sm_median': 'sm_median',
    'data_count': 'data_count',
    'data_coverage': 'data_cov',
    'invalid_count': 'invalid_ct',
    'original_range_min': 'orig_min',
    'original_range_max': 'orig_max'
}

gdf_clean = gdf.rename(columns=column_mapping)

print(f"GeoDataFrame created with {len(gdf_clean)} features")
print("\nSample of GeoDataFrame:")
print(gdf_clean.head())



=== Creating GeoDataFrame for Shapefile Export ===
GeoDataFrame created with 16 features

Sample of GeoDataFrame:
   sensor_id     network          station sensor_nm       lat      lon  \
0          0  AMMA-CATCH      Banizoumbou   CS616-1  13.53250  2.66040   
1          1  AMMA-CATCH      Banizoumbou   CS616-2  13.53250  2.66040   
2          2  AMMA-CATCH  Belefoungou-Mid     CS616   9.79506  1.70994   
3          3  AMMA-CATCH  Belefoungou-Mid     CS616   9.79506  1.70994   
4          4  AMMA-CATCH  Belefoungou-Top     CS616   9.78986  1.70994   

   elevation  depth_from  depth_to  clay_frac  ...    sm_std    sm_min  \
0      208.0        0.05      0.05        5.0  ...  0.028664  0.005450   
1      208.0        0.05      0.05        5.0  ...  0.032553  0.017350   
2      414.0        0.05      0.05       22.0  ...  0.091609  0.026675   
3      414.0        0.10      0.10       22.0  ...  0.093208  0.054354   
4      414.0        0.05      0.05       22.0  ...  0.075493  0.006929

In [40]:
# ========================================
# 6. EXPORT TO SHAPEFILE
# ========================================

print("\n=== Exporting to Shapefile ===")

# Export to shapefile
output_path = r'data\stations\amma_catch_soil_moisture.shp'
gdf_clean.to_file(output_path, driver='ESRI Shapefile')

print(f"Shapefile exported to: {output_path}")

# ========================================
# 7. SUMMARY STATISTICS
# ========================================

print("\n=== Summary Statistics ===")
print(f"Network: {gdf_clean.network.unique()}")
print(f"Number of stations: {gdf_clean.station.nunique()}")
print(f"Number of sensors: {len(gdf_clean)}")
print(f"Latitude range: {gdf_clean.lat.min():.4f} to {gdf_clean.lat.max():.4f}")
print(f"Longitude range: {gdf_clean.lon.min():.4f} to {gdf_clean.lon.max():.4f}")
print(f"Elevation range: {gdf_clean.elevation.min():.1f} to {gdf_clean.elevation.max():.1f} m")
print(f"Depth range: {gdf_clean.depth_from.min():.2f} to {gdf_clean.depth_to.max():.2f} m")
print(f"Soil moisture mean range: {gdf_clean.sm_mean.min():.4f} to {gdf_clean.sm_mean.max():.4f}")


=== Exporting to Shapefile ===
Shapefile exported to: data\stations\amma_catch_soil_moisture.shp

=== Summary Statistics ===
Network: ['AMMA-CATCH']
Number of stations: 7
Number of sensors: 16
Latitude range: 9.7441 to 13.6456
Longitude range: 1.6053 to 2.6956
Elevation range: 208.0 to 445.0 m
Depth range: 0.05 to 0.40 m
Soil moisture mean range: 0.0279 to 0.1686


In [ ]:
# ========================================
# 8. CREATE TIME SERIES EXPORT (OPTIONAL)
# ========================================

print("\n=== Creating Time Series Export (Optional) ===")

# Create a time series dataframe with daily values
time_series_data = []

# Get the number of sensors from the daily resampled data
n_sensors = daily_sm.shape[0] if len(daily_sm.shape) > 1 else daily_sm.sizes['sensor']

for sensor_idx in range(n_sensors):
    sensor_static = static_df.iloc[sensor_idx]
    sensor_sm_data = daily_sm.isel(sensor=sensor_idx)
    
    for date_idx, date_val in enumerate(sensor_sm_data.date_time.values):
        sm_val = float(sensor_sm_data.values[date_idx])
        
        time_series_data.append({
            'sensor_id': sensor_idx,
            'network': sensor_static['network'],
            'station': sensor_static['station'],
            'date': pd.to_datetime(date_val).date(),
            'soil_moisture': sm_val if not np.isnan(sm_val) else None,
            'latitude': sensor_static['latitude'],
            'longitude': sensor_static['longitude']
        })

time_series_df = pd.DataFrame(time_series_data)
time_series_df = time_series_df.dropna(subset=['soil_moisture'])  # Remove NaN values

print(f"Time series dataframe shape: {time_series_df.shape}")

# Export time series to CSV
time_series_df.to_csv('amma_catch_time_series.csv', index=False)
print("Time series data exported to: amma_catch_time_series.csv")



=== Creating Time Series Export (Optional) ===


In [ ]:

# ========================================
# 9. VISUALIZATION (OPTIONAL)
# ========================================

print("\n=== Creating Visualization ===")

import matplotlib.pyplot as plt

# Create a simple map of sensor locations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Sensor locations
scatter = ax1.scatter(gdf_clean.lon, gdf_clean.lat, 
                     c=gdf_clean.sm_mean, cmap='Blues', 
                     s=100, alpha=0.7, edgecolors='black')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('AMMA-CATCH Sensor Locations\n(Colored by Mean Soil Moisture)')
plt.colorbar(scatter, ax=ax1, label='Mean Soil Moisture')

# Plot 2: Soil moisture statistics
ax2.boxplot([gdf_clean.sm_mean.dropna(), gdf_clean.sm_std.dropna()], 
            labels=['Mean', 'Std Dev'])
ax2.set_ylabel('Soil Moisture')
ax2.set_title('Soil Moisture Statistics Distribution')

plt.tight_layout()
plt.savefig('amma_catch_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n=== Processing Complete ===")
print("Files created:")
print("1. amma_catch_soil_moisture.shp (and associated files)")
print("2. amma_catch_time_series.csv")
print("3. amma_catch_summary.png")